In [15]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn import functional as F


In [20]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 8
batch_size = 100
learning_rate = 0.01

In [3]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./files', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=False)

test_dataset = dsets.MNIST(root='./files', 
                           train=False, 
                           transform=transforms.ToTensor(),
                           download=False)

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [13]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


## MODEL

In [16]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = F.sigmoid(self.linear(x))
        return out

In [17]:
#Instantiate the model class 
model = LogisticRegression(input_size, num_classes)

# Loss: Computes Softmax and then CrossEntropy loss
criterion = nn.CrossEntropyLoss()  

#Optimizer: Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [24]:
training_losses = []

# Training the Model: If you Rerun this cell, model start training from where you left it (Weights)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        #Reset the Gradients before Training 
        optimizer.zero_grad()
        outputs = model(images)
        #Find loss using predicted output and true labels
        loss = criterion(outputs, labels)
        #BackProp the grads
        loss.backward()
        #Update the parameteres
        optimizer.step()
        
        training_losses.append(loss.data)
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data))



/home/strider/anaconda3/envs/fai/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: [1/8], Step: [100/600], Loss: 1.9667
Epoch: [1/8], Step: [200/600], Loss: 1.9632
Epoch: [1/8], Step: [300/600], Loss: 1.9607
Epoch: [1/8], Step: [400/600], Loss: 1.9641
Epoch: [1/8], Step: [500/600], Loss: 1.9597
Epoch: [1/8], Step: [600/600], Loss: 1.9193
Epoch: [2/8], Step: [100/600], Loss: 1.9112
Epoch: [2/8], Step: [200/600], Loss: 1.9630
Epoch: [2/8], Step: [300/600], Loss: 1.9537
Epoch: [2/8], Step: [400/600], Loss: 1.9627
Epoch: [2/8], Step: [500/600], Loss: 1.9264
Epoch: [2/8], Step: [600/600], Loss: 1.9449
Epoch: [3/8], Step: [100/600], Loss: 1.9434
Epoch: [3/8], Step: [200/600], Loss: 1.9346
Epoch: [3/8], Step: [300/600], Loss: 1.9304
Epoch: [3/8], Step: [400/600], Loss: 1.9276
Epoch: [3/8], Step: [500/600], Loss: 1.9521
Epoch: [3/8], Step: [600/600], Loss: 1.9475
Epoch: [4/8], Step: [100/600], Loss: 1.9515
Epoch: [4/8], Step: [200/600], Loss: 1.9405
Epoch: [4/8], Step: [300/600], Loss: 1.9353
Epoch: [4/8], Step: [400/600], Loss: 1.9123
Epoch: [4/8], Step: [500/600], L

In [33]:
training_losses

[tensor(1.9658),
 tensor(1.9626),
 tensor(1.9452),
 tensor(1.9597),
 tensor(1.9604),
 tensor(1.9498),
 tensor(1.9490),
 tensor(1.9619),
 tensor(1.9611),
 tensor(1.9612),
 tensor(1.9730),
 tensor(1.9487),
 tensor(1.9656),
 tensor(1.9497),
 tensor(1.9679),
 tensor(1.9556),
 tensor(1.9602),
 tensor(1.9348),
 tensor(1.9422),
 tensor(1.9422),
 tensor(1.9509),
 tensor(1.9641),
 tensor(1.9707),
 tensor(1.9658),
 tensor(1.9476),
 tensor(1.9667),
 tensor(1.9268),
 tensor(1.9384),
 tensor(1.9429),
 tensor(1.9678),
 tensor(1.9424),
 tensor(1.9471),
 tensor(1.9599),
 tensor(1.9604),
 tensor(1.9813),
 tensor(1.9395),
 tensor(1.9455),
 tensor(1.9375),
 tensor(1.9531),
 tensor(1.9485),
 tensor(1.9890),
 tensor(1.9667),
 tensor(1.9577),
 tensor(1.9640),
 tensor(1.9700),
 tensor(1.9356),
 tensor(1.9309),
 tensor(1.9490),
 tensor(1.9289),
 tensor(1.9528),
 tensor(1.9450),
 tensor(1.9370),
 tensor(1.9669),
 tensor(1.9480),
 tensor(1.9334),
 tensor(1.9749),
 tensor(1.9476),
 tensor(1.9702),
 tensor(1.9713

In [27]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

/home/strider/anaconda3/envs/fai/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy of the model on the 10000 test images: 81 %


In [28]:
torch.save(model.state_dict(), './results/model_linear_regression.pth')
torch.save(optimizer.state_dict(), './results/optimizer_linear_regression.pth')